In [ ]:
# import libraries
import os
import pandas as pd
from vanna.ollama import Ollama
from vanna.chromadb import ChromaDB_VectorStore

In [ ]:
# setup directory and filepath
cdir = os.getcwd()
chroma = os.path.join(cdir, "database")
database = os.path.join(cdir, "database.sqlite")
training = os.path.join(cdir, "training", "dataset.csv")

In [ ]:
# instantiate vanna
class Vanna(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None) -> None:
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)


vn = Vanna(
    config={
        "model": "qwen2.5:3b",
        "path": chroma,
    }
)

In [ ]:
# connect to database
vn.connect_to_sqlite(database)

In [ ]:
# training with ddls
ddls = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")
for ddl in ddls["sql"].to_list():
    vn.train(ddl=ddl)

In [ ]:
# training with questions and answer pairs
df = pd.read_csv(training, delimiter=";")

for index, row in df.iterrows():
    question = row["question"]
    answer = row["answer"] + ";"
    vn.train(question=question, sql=answer)